In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05188393592834473
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 5.347264089380393
200000 10.020335821958142
300000 10.9057387230974
400000 12.336888771214834
500000 12.288601437911652
600000 13.013628563433143
700000 12.193961210990935
800000 13.938595632756112
900000 18.15561917824068
1000000 20.3057106405687
1100000 18.964058803735234
1200000 10.194052226748868
1300000 23.98840407011346
1400000 27.017283480441886
1500000 26.669864062316197
1600000 23.546374218002082
1700000 26.391557387883957
1800000 27.284809907889507
1900000 27.084564392039276
2000000 22.95487377036308
2100000 26.499677837760586
2200000 27.21376938490077
2300000 7.299841399490783
2400000 28.522154654623222
2500000 32.70436911369681
2600000 34.70282277324545
2700000 31.60693126831278
2800000 29.161284188279986
2900000 26.898853070569732
3000000 27.97

29400000 23.96381642079705
29500000 8.035425044608443
29600000 22.632962104677315
29700000 24.48119747487964
29800000 21.398504063723326
29900000 23.868996511658324
30000000 21.468392286929596
30100000 22.745547356553963
30200000 20.919389460676904
30300000 21.450388065675497
30400000 5.653585805532456
30500000 20.27629468331389
30600000 21.839756536180975
30700000 20.96363438263773
30800000 22.16065876257084
30900000 21.511709213171397
31000000 22.25290994043844
31100000 21.92088667969464
31200000 22.99659561964303
31300000 8.957716928305537
31400000 22.443925689799123
31500000 21.60838694654912
31600000 23.302498745115194
31700000 18.46259801736644
31800000 10.689183778087347
31900000 22.95102510946163
32000000 20.82675540157195
32100000 23.053759212806714
32200000 19.37975368006271
32300000 21.56162304637699
32400000 19.826809423647546
32500000 19.62662736069582
32600000 12.951022434418569
32700000 13.503104480342003
32800000 20.574802777287047
32900000 19.145705081524596
33000000 1

59200000 17.857610846548408
59300000 15.95195508320349
59400000 17.7253175920995
59500000 16.15785025894041
59600000 7.461407660740153
59700000 15.712765207788078
59800000 16.616937858540762
59900000 17.328888446278146
60000000 16.16347967142336
60100000 18.083621548610925
60200000 17.256762079460593
60300000 7.507771285132372
60400000 19.059986651994635
60500000 18.330837040905962
60600000 19.296733184814038
60700000 17.984876324002744
60800000 20.06574842855943
60900000 19.04618847350569
61000000 18.828319486310868
61100000 19.729267524914803
61200000 8.641229866657712
61300000 17.305290689991804
61400000 17.42576046337787
61500000 17.466292185936187
61600000 17.35634785932356
61700000 17.138618132998136
61800000 14.885387877357518
61900000 8.805794559334874
62000000 17.154424662745402
62100000 16.99262092189145
62200000 16.104290141686548
62300000 17.673927332321323
62400000 17.50765308120046
62500000 9.829948833176285
62600000 16.81275923339444
62700000 18.78337092788841
62800000 1

88900000 10.564620140147298
89000000 15.262131931363783
89100000 15.02460173530004
89200000 15.116640563857358
89300000 6.605075893916031
89400000 15.391482778105107
89500000 15.542502851070166
89600000 16.329926916949525
89700000 16.191094754370347
89800000 16.514423885035903
89900000 16.343982458393295
90000000 16.95937259263156
90100000 16.554691047100743
90200000 16.991750361505296
90300000 16.904184349137022
90400000 16.28271637958972
90500000 15.548367785851497
90600000 12.596324685804781
90700000 8.729459340106128
90800000 14.858358299885817
90900000 14.639106287114236
91000000 15.500720239738266
91100000 4.21616878336411
91200000 15.735514165454187
91300000 15.140502204761834
91400000 14.891401965484315
91500000 8.22041828305467
91600000 13.53306181331152
91700000 15.159300110380615
91800000 15.832493063944518
91900000 16.508827928628214
92000000 16.697037968210314
92100000 16.02420263450223
92200000 16.631755336846343
92300000 16.443161716160017
92400000 16.60485765751043
9250

117900000 8.689873686984132
118000000 10.801225954904872
118100000 5.708197393529746
118200000 12.454117176725136
118300000 5.715758352531374
118400000 13.121470411685033
118500000 12.802899712345795
118600000 13.25671281286812
118700000 12.631922632797023
118800000 12.989876588485616
118900000 13.101297360791962
119000000 13.985647557162615
119100000 14.460589827331486
119200000 14.24012394786016
119300000 15.04264051403857
119400000 14.169313220991912
119500000 13.571824550246447
119600000 13.577455936791841
119700000 13.413853084346751
119800000 13.27185676495251
119900000 12.829507562318037
120000000 13.37764211005048
120100000 5.932968950847606
120200000 12.515271989659402
120300000 5.218430148302677
120400000 11.668000020212471
120500000 7.698821477699551
120600000 11.468723551361405
120700000 13.427758656055461
120800000 13.185530074469352
120900000 12.974992011533324
121000000 13.375230860586434
121100000 13.032613171535349
121200000 13.59815712338399
121300000 14.1896637445495

...

175500000 9.373468946960267
175600000 9.244957205187797
175700000 8.75537514689114
175800000 8.69304951239464
175900000 8.780360773995966
176000000 8.5700625205741
176100000 8.819363166152252
176200000 8.363253926782965
176300000 3.7749581878092386
176400000 8.584043096291175
176500000 3.7808107756057785
176600000 7.442716817443304
176700000 5.568360635899842
176800000 6.2052692165415095
176900000 8.491762107475353
177000000 8.43905565127387
177100000 8.564710940764963
177200000 8.421922451647404
177300000 8.86151221209443
177400000 8.868505762048125
177500000 9.468587327744162
177600000 9.523813166697835
177700000 9.585697390798579
177800000 9.329983111963678
177900000 8.980840729020708
178000000 8.826927989785196
178100000 8.84911874062615
178200000 9.027932921444277
178300000 8.50931795945716
178400000 8.49953792857437
178500000 5.237182755824278
178600000 7.404513329214353
178700000 7.1684775949671655
178800000 4.019586191900779
178900000 8.61212499130267
179000000 3.76140940415726

204800000 7.326311864816918
204900000 6.968329927358432
205000000 7.019320184080839
205100000 6.823924734463218
205200000 6.616483498186359
205300000 2.7262062180704874
205400000 6.743414537154086
205500000 6.504470152449879
205600000 6.559319531633397
205700000 4.412351024017665
205800000 4.57739174962649
205900000 6.513408913788169
206000000 6.675967897339103
206100000 6.6084266343414315
206200000 2.4352008599806823
206300000 6.707738990501083
206400000 6.771199385509672
206500000 7.0004409117675825
206600000 7.1166875528050975
206700000 7.535444676061521
206800000 7.120221875557334
206900000 7.0346118457153075
207000000 7.103177024629229
207100000 7.027555134830832
207200000 6.718057536661853
207300000 6.5664412021180665
207400000 6.260798578652084
207500000 3.8729821438260075
207600000 5.28043822619217
207700000 6.206530246540547
207800000 6.11614583320369
207900000 6.430875793721928
208000000 1.8236121622121433
208100000 6.418454008632743
208200000 6.186972495914972
208300000 6.12

234000000 5.0668262012338
234100000 5.407885520114888
234200000 4.918612422802394
234300000 2.0446311562304067
234400000 5.101638066067577
234500000 4.707821272139616
234600000 4.732302219275345
234700000 4.4249715967664125
234800000 4.6058052465096875
234900000 3.983059313270112
235000000 2.087767565545976
235100000 4.586034200064421
235200000 4.36184388709153
235300000 4.59665914830634
235400000 4.534764704571671
235500000 4.837145431824061
235600000 3.4212475948417485
235700000 3.7696680067476227
235800000 5.7157234074403815
235900000 5.2514621560249335
236000000 5.22668126555604
236100000 5.557792497912642
236200000 5.009100662778228
236300000 5.168483324884133
236400000 4.854130711508956
236500000 2.990140039587051
236600000 3.9980873082145054
236700000 4.721045179883504
236800000 4.550876923222313
236900000 4.340745173603071
237000000 4.506602881978217
237100000 4.1830636741468705
237200000 1.8868656930022174
237300000 4.005070963736488
237400000 4.3038594929816725
237500000 4.52

263000000 2.8795003300722106
263100000 2.8516606546986965
263200000 2.7182226181892353
263300000 1.0659675119212966
263400000 2.669046581276218
263500000 2.7657688094540536
263600000 2.6117587105845943
263700000 2.7198694779940875
263800000 2.552219375201225
263900000 2.616869522041771
264000000 2.387279597204301
264100000 2.56811214222003
264200000 0.7297134108730357
264300000 2.5152936921936537
264400000 2.481985032819182
264500000 2.504207162122554
264600000 2.7019230062828727
264700000 2.52123463318939
264800000 2.769612782945282
264900000 2.452989450704343
265000000 2.8269964573550923
265100000 0.8878320439105333
265200000 2.495287924293816
265300000 2.7599210647075174
265400000 2.4420607825965615
265500000 1.7371051312828794
265600000 1.9434521538886729
265700000 2.4208571917250588
265800000 2.591986540653765
265900000 2.2686655372915467
266000000 2.5739437923132713
266100000 2.2428839041703243
266200000 2.467637490302056
266300000 2.07413450111866
266400000 1.5358286277575328
26

291500000 0.31748875804494486
291600000 0.3493196613917814
291700000 0.30878372056679926
291800000 0.354387900279012
291900000 0.3693706126536395
292000000 0.2134581287291646
292100000 0.33668642276047
292200000 0.20081688282104332
292300000 0.047580000476926565
292400000 0.20249649448770285
292500000 0.19609439318902624
292600000 0.14770303074626126
292700000 0.19218282518519292
292800000 0.20890720082058842
292900000 0.19846641217351954
293000000 0.21490761495795516
293100000 0.19579549770686364
293200000 0.20698110950312018
293300000 0.15259280178094572
293400000 0.06704827379096548
293500000 0.10036193442551296
293600000 0.12911812045898702
293700000 0.11532215595539741
293800000 0.1258183963700533
293900000 0.18337428607002232
294000000 0.11366103876135748
294100000 0.11598178166878223
294200000 0.10823788715448975
294300000 0.13502090847185916
294400000 0.1050119662915965
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.06962123562757837
29460000